THIS SCRIPT PREPROCESSES THE CONSOLIDATED DATA COLLECTED AND CREATES SEPERATE TABLES FOR ANALYSIS

In [1]:
# Dependencies
from config import homeDepot_API
from serpapi import GoogleSearch
import pandas as pd
import numpy as np
from openpyxl import load_workbook

STEP 1: PREPROCESS DATA

In [2]:
# Collect all data

path0 = "C:\\Users\\kijah\\Documents\\Data_Bootcamp\\Estimator_Project\\HistoricalPrices.csv"
raw_prices = pd.read_csv(path0)

path = "C:\\Users\\kijah\\Documents\\Data_Bootcamp\\Estimator_Project\\data.csv"
data = pd.read_csv(path)

data.dtypes

material_description     object
material_price          float64
vendor                   object
location                 object
formatted_datetime       object
dow                      object
dtype: object

In [3]:
# Create Regional columns

# Split location column to create a zip code column

data['zip'] = (np.where(data['location'].str.contains(','), data['location'].str.split(',').str[-1], data['location']))
data['zip'] = data['zip'].str.strip()
data['zip'] = (np.where(data['zip'].str.len()>5, "other", data['zip']))

region = []
for i in data['zip']:
    if i.startswith("0"):
        region.append('northeast')
    elif i.startswith("1"):
        region.append('northeast')
    elif i.startswith("2"):
        region.append('east')
    elif i.startswith("3"):
        region.append('southeast')
    elif i.startswith("4"):
        region.append('midwest')
    elif i.startswith("5"):
        region.append('northplaines')
    elif i.startswith("6"):
        region.append('central')
    elif i.startswith("7"):
        region.append('south')
    elif i.startswith("8"):
        region.append('mountains')
    elif i.startswith("9"):
        region.append('west')
    else:
        region.append('other')
        
        
data['region'] = region
data.formatted_datetime = pd.to_datetime(data.formatted_datetime)
data['date'] = data.formatted_datetime.dt.date
data.date = pd.to_datetime(data.date)
data.drop(columns =['formatted_datetime'], inplace = True)

In [4]:
data.dtypes

material_description            object
material_price                 float64
vendor                          object
location                        object
dow                             object
zip                             object
region                          object
date                    datetime64[ns]
dtype: object

In [5]:
raw_prices.date = pd.to_datetime(raw_prices.date)
raw_prices = raw_prices.iloc[:,[0,1,2]]
raw_prices

,date,steel_price,copper_price
0,2022-11-13,1700.39,8062.50
1,2022-11-12,1700.39,8062.50
2,2022-11-11,1700.39,8062.50
3,2022-11-10,1630.76,8062.50
4,2022-11-09,1583.59,8033.85
5,2022-11-08,1624.27,7973.00
6,2022-11-07,1600.95,7953.85
7,2022-11-06,1600.95,7953.85
8,2022-11-05,1600.95,7953.85
9,2022-11-04,1637.04,7902.35


In [6]:
data = pd.merge(data, raw_prices)
data.head()

,material_description,material_price,vendor,location,dow,zip,region,date,steel_price,copper_price
0,1/2 in. x 10 ft. Electric Metallic Tube (EMT) ...,7.22,homedepot,"2455 Paces Ferry Rd SE. Atlanta, GA, 30339",Thursday,30339,southeast,2022-10-20,1500.53,7585.5
1,1/2 in. x 10 ft. Electric Metallic Tube (EMT) ...,5.99,lowes,"1000 Lowe's Blvd. Mooresville, NC, 28117",Thursday,28117,east,2022-10-20,1500.53,7585.5
2,1/2 in. x 10 ft. Electric Metallic Tube (EMT) ...,4.73,gordonelectricsupply,"1290 N Hobbie Ave. Kankakee, Illinois, 60901",Thursday,60901,central,2022-10-20,1500.53,7585.5
3,1/2 in. x 10 ft. Electric Metallic Tube (EMT) ...,5.88,supplyworks,"6260 Abbott Dr. Omaha, Nebraska, 68110",Thursday,68110,central,2022-10-20,1500.53,7585.5
4,1/2 in. x 10 ft. Electric Metallic Tube (EMT) ...,9.09,ebay,"2025 Hamilton Avenue. San Jose, California, 95125",Thursday,95125,west,2022-10-20,1500.53,7585.5


In [7]:
# Creating 'material_description' by extracting from Home Depot. Home Depot's product id is what is used to query

material_name = []

ids = ['100400409', '100400406', '100137321', '100400405', 
       '202241090', '100144234', '202077145', '202077147', 
       '202288508', '202304774', '202304649', '202304645', 
       '202019377', '202316275', '202019375', '202316276']

for i in ids:

    params = {
      "engine": "home_depot_product",
      "product_id": i,
      "api_key": homeDepot_API
    }

    search = GoogleSearch(params)
    results = search.get_dict()
    products = results['product_results']
    
    material_name.append(products['title'])
        
material_description = pd.DataFrame(data={'material_description': material_name}).iloc[[3,1,0,5,8,4,2,6,7,12,13,14,15,9,10,11],:].reset_index(drop=True)
material_description

https://serpapi.com/search
https://serpapi.com/search
https://serpapi.com/search
https://serpapi.com/search
https://serpapi.com/search
https://serpapi.com/search
https://serpapi.com/search
https://serpapi.com/search
https://serpapi.com/search
https://serpapi.com/search
https://serpapi.com/search
https://serpapi.com/search
https://serpapi.com/search
https://serpapi.com/search
https://serpapi.com/search
https://serpapi.com/search


,material_description
0,1/2 in. x 10 ft. Electric Metallic Tube (EMT) ...
1,3/4 in. x 10 ft. Electric Metallic Tube (EMT) ...
2,1 in. x 10 ft. Electric Metallic Tube (EMT) Co...
3,1/2 in. Electric Metallic Tube (EMT) Set-Screw...
4,3/4 in. Standard Fitting Electric Metallic Tub...
5,1 in. Electric Metallic Tube (EMT) Set-Screw C...
6,1/2 in. Electrical Metallic Tube (EMT) Set-Scr...
7,3/4 in. Electrical Metallic Tube (EMT) Set-Scr...
8,1 in. Electrical Metallic Tube (EMT) Set-Screw...
9,250 ft. 14/2 Romex SIMpull Solid NM-B W/G Wire


In [8]:
# Creating a dataframe for each material item

R = material_description['material_description']

emt12 = data[data['material_description']==R[0]].reset_index(drop=True)
d0 = pd.DataFrame(data={'emt12': emt12['material_price'].describe()})

emt34 = data[data['material_description']==R[1]].reset_index(drop=True)
d1 = pd.DataFrame(data={'emt34': emt34['material_price'].describe()})

emt1 = data[data['material_description']==R[2]].reset_index(drop=True)
d2 = pd.DataFrame(data={'emt1': emt1['material_price'].describe()})

con12 = data[data['material_description']==R[3]].reset_index(drop=True)
d3 = pd.DataFrame(data={'con12': con12['material_price'].describe()})

con34 = data[data['material_description']==R[4]].reset_index(drop=True)
d4 = pd.DataFrame(data={'con34': con34['material_price'].describe()})

con1 = data[data['material_description']==R[5]].reset_index(drop=True)
d5 = pd.DataFrame(data={'con1': con1['material_price'].describe()})

cup12 = data[data['material_description']==R[6]].reset_index(drop=True)
d6 = pd.DataFrame(data={'cup12': cup12['material_price'].describe()})

cup34 = data[data['material_description']==R[7]].reset_index(drop=True)
d7 = pd.DataFrame(data={'cup34': cup34['material_price'].describe()})

cup1 = data[data['material_description']==R[8]].reset_index(drop=True)
d8 = pd.DataFrame(data={'cup1': cup1['material_price'].describe()})

rom142 = data[data['material_description']==R[9]].reset_index(drop=True)
d9 = pd.DataFrame(data={'rom142': rom142['material_price'].describe()})

rom143 = data[data['material_description']==R[10]].reset_index(drop=True)
d10 = pd.DataFrame(data={'rom143': rom143['material_price'].describe()})

rom122 = data[data['material_description']==R[11]].reset_index(drop=True)
d11 = pd.DataFrame(data={'rom122': rom122['material_price'].describe()})

rom123 = data[data['material_description']==R[12]].reset_index(drop=True)
d12 = pd.DataFrame(data={'rom123': rom123['material_price'].describe()})

wir4 = data[data['material_description']==R[13]].reset_index(drop=True)
d13 = pd.DataFrame(data={'wir4': wir4['material_price'].describe()})

wir6 = data[data['material_description']==R[14]].reset_index(drop=True)
d14 = pd.DataFrame(data={'wir6': wir6['material_price'].describe()})

wir10 = data[data['material_description']==R[15]].reset_index(drop=True)
d15 = pd.DataFrame(data={'wir10': wir10['material_price'].describe()})

In [9]:
# descriptive_table1 = data.groupby('material_description').describe()['material_price'].transpose()
# descriptive_table1

In [10]:
# # order in code should match up with R
# R = material_description['material_description']
# mat_name = ['emt12',
# 'emt34',
# 'emt1',
# 'cup12',
# 'cup34',
# 'cup1',
# 'con12',
# 'con34',
# 'con1',
# 'rom142',
# 'rom143',
# 'rom122',
# 'rom123',
# 'wir4',
# 'wir6',
# 'wir10',
# 'total']

# for i, desc in enumerate(R):
#     descriptive_table1.rename({desc: mat_name[i]}, axis=1)
# descriptive_table1

In [11]:
# Creating a descriptive table for each item

descriptive_table = d0.join([d1,d2,d3,d4,d5,d6,d7,d8,d9,d10,d11,d12,d13,d14,d15])
descriptive_table

,emt12,emt34,emt1,con12,con34,con1,cup12,cup34,cup1,rom142,rom143,rom122,rom123,wir4,wir6,wir10
count,93.000000,69.000000,60.000000,52.000000,73.000000,23.000000,59.000000,58.000000,26.000000,95.000000,68.000000,50.000000,62.000000,61.000000,75.000000,68.000000
mean,6.713011,10.348261,20.847333,4.857500,3.774521,1.199565,5.469661,7.354655,1.636154,108.403053,172.568382,158.172800,300.690000,771.422623,450.012400,174.565735
std,3.248268,1.529384,7.323187,2.116757,1.209735,0.237706,2.131372,4.604312,0.195838,12.624971,43.183500,20.400919,21.596331,177.698314,124.423091,64.058389
min,4.590000,8.120000,8.900000,2.100000,2.980000,1.150000,2.100000,4.600000,0.980000,84.990000,135.000000,149.000000,279.000000,613.640000,344.990000,145.000000
25%,5.880000,8.600000,17.550000,3.980000,2.980000,1.150000,3.980000,4.600000,1.600000,110.535000,158.000000,149.000000,289.000000,614.280000,409.000000,155.990000
50%,5.990000,10.470000,20.620000,3.980000,2.980000,1.150000,5.990000,5.990000,1.600000,114.000000,159.000000,149.000000,289.000000,709.000000,445.810000,156.000000
75%,7.220000,11.910000,24.990000,5.990000,3.660000,1.150000,5.990000,6.450000,1.600000,117.000000,160.590000,149.000000,315.010000,1047.780000,470.430000,165.000000
max,26.180000,12.680000,34.230000,8.540000,5.990000,2.290000,9.990000,20.000000,1.990000,128.620000,330.850000,227.290000,352.780000,1047.780000,1144.690000,398.950000


In [12]:
# def check_price(price, item_name):
#     low = described_groups.loc['25%', item_name]
#     high = described_groups.loc['75%', item_name]
    
#     if price <= low:
#         return 'low price'
#     elif price >= high:
#         return 'high price'
#     else:
#         return 'about average'
# data['target_price'] = data.apply(lambda x: check_price(x['matierla_price'], x['material_description'], axis=1)

In [13]:
# Add target price column to material item dataframes


tp0 = []
t0 = emt12['material_price']
for i in t0:
    if i <= descriptive_table["emt12"][4]:        
        tp0.append('low price')
    elif i >= descriptive_table["emt12"][6]:        
        tp0.append('high price')
    else:        
        tp0.append('about average')
emt12['target price'] = tp0

tp1 = []
t1 = emt34['material_price']
for i in t1:
    if i <= descriptive_table["emt34"][4]:        
        tp1.append('low price')
    elif i >= descriptive_table["emt34"][6]:        
        tp1.append('high price')
    else:        
        tp1.append('about average')        
emt34['target price'] = tp1

tp2 = []
t2 = emt1['material_price']
for i in t2:
    if i <= descriptive_table["emt1"][4]:        
        tp2.append('low price')
    elif i >= descriptive_table["emt1"][6]:        
        tp2.append('high price')
    else:        
        tp2.append('about average')
emt1['target price'] = tp2

tp3 = []
t3 = cup12['material_price']
for i in t3:
    if i <= descriptive_table["cup12"][4]:        
        tp3.append('low price')
    elif i >= descriptive_table["cup12"][6]:        
        tp3.append('high price')
    else:        
        tp3.append('about average')
cup12['target price'] = tp3

tp4 = []
t4 = cup34['material_price']
for i in t4:
    if i <= descriptive_table["cup34"][4]:        
        tp4.append('low price')
    elif i >= descriptive_table["cup34"][6]:        
        tp4.append('high price')
    else:        
        tp4.append('about average')
cup34['target price'] = tp4

tp5 = []
t5 = cup1['material_price']
for i in t5:
    if i <= descriptive_table["cup1"][4]:        
        tp5.append('low price')
    elif i >= descriptive_table["cup1"][6]:        
        tp5.append('high price')
    else:        
        tp5.append('about average')
cup1['target price'] = tp5

tp6 = []
t6 = con12['material_price']
for i in t6:
    if i <= descriptive_table["con12"][4]:        
        tp6.append('low price')
    elif i >= descriptive_table["con12"][6]:        
        tp6.append('high price')
    else:        
        tp6.append('about average')
con12['target price'] = tp6

tp7 = []
t7 = con34['material_price']
for i in t7:
    if i <= descriptive_table["con34"][4]:        
        tp7.append('low price')
    elif i >= descriptive_table["con34"][6]:        
        tp7.append('high price')
    else:        
        tp7.append('about average')
con34['target price'] = tp7

tp8 = []
t8 = con1['material_price']
for i in t8:
    if i <= descriptive_table["con1"][4]:        
        tp8.append('low price')
    elif i >= descriptive_table["con1"][6]:        
        tp8.append('high price')
    else:        
        tp8.append('about average')
con1['target price'] = tp8

tp9 = []
t9 = rom142['material_price']
for i in t9:
    if i <= descriptive_table["rom142"][4]:        
        tp9.append('low price')
    elif i >= descriptive_table["rom142"][6]:        
        tp9.append('high price')
    else:        
        tp9.append('about average')
rom142['target price'] = tp9

tp10 = []
t10 = rom143['material_price']
for i in t10:
    if i <= descriptive_table["rom143"][4]:        
        tp10.append('low price')
    elif i >= descriptive_table["rom143"][6]:        
        tp10.append('high price')
    else:        
        tp10.append('about average')
rom143['target price'] = tp10

tp11 = []
t11 = rom122['material_price']
for i in t11:
    if i <= descriptive_table["rom122"][4]:        
        tp11.append('low price')
    elif i >= descriptive_table["rom122"][6]:        
        tp11.append('high price')
    else:        
        tp11.append('about average')        
rom122['target price'] = tp11

tp12 = []
t12 = rom123['material_price']
for i in t12:
    if i <= descriptive_table["rom123"][4]:        
        tp12.append('low price')
    elif i >= descriptive_table["rom123"][6]:        
        tp12.append('high price')
    else:        
        tp12.append('about average')
rom123['target price'] = tp12

tp13 = []
t13 = wir4['material_price']
for i in t13:
    if i <= descriptive_table["wir4"][4]:        
        tp13.append('low price')
    elif i >= descriptive_table["wir4"][6]:        
        tp13.append('high price')
    else:        
        tp13.append('about average')
wir4['target price'] = tp13

tp14 = []
t14 = wir6['material_price']
for i in t14:
    if i <= descriptive_table["wir6"][4]:        
        tp14.append('low price')
    elif i >= descriptive_table["wir6"][6]:        
        tp14.append('high price')
    else:        
        tp14.append('about average')
wir6['target price'] = tp14

tp15 = []
t15 = wir10['material_price']
for i in t15:
    if i <= descriptive_table["wir10"][4]:        
        tp15.append('low price')
    elif i >= descriptive_table["wir10"][6]:        
        tp15.append('high price')
    else:        
        tp15.append('about average')
wir10['target price'] = tp15



In [14]:
frames = [emt12, emt34, emt1, cup12, cup34, cup1, con12, con34, con1, rom142, rom143, rom122, rom123, wir4, wir6, wir10,]
preprocessed_df = pd.concat(frames)
preprocessed_df.head(10)

,material_description,material_price,vendor,location,dow,zip,region,date,steel_price,copper_price,target price
0,1/2 in. x 10 ft. Electric Metallic Tube (EMT) ...,7.22,homedepot,"2455 Paces Ferry Rd SE. Atlanta, GA, 30339",Thursday,30339,southeast,2022-10-20,1500.53,7585.5,high price
1,1/2 in. x 10 ft. Electric Metallic Tube (EMT) ...,5.99,lowes,"1000 Lowe's Blvd. Mooresville, NC, 28117",Thursday,28117,east,2022-10-20,1500.53,7585.5,about average
2,1/2 in. x 10 ft. Electric Metallic Tube (EMT) ...,4.73,gordonelectricsupply,"1290 N Hobbie Ave. Kankakee, Illinois, 60901",Thursday,60901,central,2022-10-20,1500.53,7585.5,low price
3,1/2 in. x 10 ft. Electric Metallic Tube (EMT) ...,5.88,supplyworks,"6260 Abbott Dr. Omaha, Nebraska, 68110",Thursday,68110,central,2022-10-20,1500.53,7585.5,low price
4,1/2 in. x 10 ft. Electric Metallic Tube (EMT) ...,9.09,ebay,"2025 Hamilton Avenue. San Jose, California, 95125",Thursday,95125,west,2022-10-20,1500.53,7585.5,high price
5,1/2 in. x 10 ft. Electric Metallic Tube (EMT) ...,7.22,homedepot,"2455 Paces Ferry Rd SE. Atlanta, GA, 30339",Friday,30339,southeast,2022-10-21,1580.45,7543.0,high price
6,1/2 in. x 10 ft. Electric Metallic Tube (EMT) ...,5.99,lowes,"1000 Lowe's Blvd. Mooresville, NC, 28117",Friday,28117,east,2022-10-21,1580.45,7543.0,about average
7,1/2 in. x 10 ft. Electric Metallic Tube (EMT) ...,4.73,gordonelectricsupply,"1290 N Hobbie Ave. Kankakee, Illinois, 60901",Friday,60901,central,2022-10-21,1580.45,7543.0,low price
8,1/2 in. x 10 ft. Electric Metallic Tube (EMT) ...,5.88,supplyworks,"6260 Abbott Dr. Omaha, Nebraska, 68110",Friday,68110,central,2022-10-21,1580.45,7543.0,low price
9,1/2 in. x 10 ft. Electric Metallic Tube (EMT) ...,7.22,homedepot,"2455 Paces Ferry Rd SE. Atlanta, GA, 30339",Saturday,30339,southeast,2022-10-22,1545.64,7540.0,high price


In [15]:
# Send Excell copies to my computer
# xlFile = "C:\\Users\\kijah\\Documents\\Data_Bootcamp\\Estimator_Project\\PreprocessedDataFrames\\data_collection.xlsx"

# book = load_workbook(xlFile)
# writer = pd.ExcelWriter(xlFile, engine = 'openpyxl')
# writer.book = book

# emt12.to_excel(writer, sheet_name='emt12')
# emt34.to_excel(writer, sheet_name='emt34')
# emt1.to_excel(writer, sheet_name='emt1')
# cup12.to_excel(writer, sheet_name='cup12')
# cup34.to_excel(writer, sheet_name='cup34')
# cup1.to_excel(writer, sheet_name='cup1')
# con12.to_excel(writer, sheet_name='con12')
# con34.to_excel(writer, sheet_name='con34')
# con1.to_excel(writer, sheet_name='con1')
# rom142.to_excel(writer, sheet_name='rom142')
# rom143.to_excel(writer, sheet_name='rom143')
# rom122.to_excel(writer, sheet_name='rom122')
# rom123.to_excel(writer, sheet_name='rom123')
# wir4.to_excel(writer, sheet_name='wir4')
# wir6.to_excel(writer, sheet_name='wir6')
# wir10.to_excel(writer, sheet_name='wir10')
# preprocessed_df.to_excel(writer, sheet_name='preprocessed_df')
# descriptive_table.to_excel(writer, sheet_name='descriptive_table')
# writer.save()
# writer.close()

# Send CSVs to my computer
preprocessed = "C:\\Users\\kijah\\Documents\\Data_Bootcamp\\Estimator_Project\\PreprocessedDataFrames\\preprocessed_df.csv"
preprocessed_df.to_csv(preprocessed, index=False)

descriptive = "C:\\Users\\kijah\\Documents\\Data_Bootcamp\\Estimator_Project\\PreprocessedDataFrames\\descriptive_table.csv"
descriptive_table.to_csv(descriptive, index=False)

emt12_df = "C:\\Users\\kijah\\Documents\\Data_Bootcamp\\Estimator_Project\\PreprocessedDataFrames\\emt12.csv"
emt12.to_csv(emt12_df, index=False)

emt34_df = "C:\\Users\\kijah\\Documents\\Data_Bootcamp\\Estimator_Project\\PreprocessedDataFrames\\emt34.csv"
emt34.to_csv(emt34_df, index=False)

emt1_df = "C:\\Users\\kijah\\Documents\\Data_Bootcamp\\Estimator_Project\\PreprocessedDataFrames\\emt1.csv"
emt1.to_csv(emt1_df, index=False)

cup12_df = "C:\\Users\\kijah\\Documents\\Data_Bootcamp\\Estimator_Project\\PreprocessedDataFrames\\cup12.csv"
cup12.to_csv(cup12_df, index=False)

cup34_df = "C:\\Users\\kijah\\Documents\\Data_Bootcamp\\Estimator_Project\\PreprocessedDataFrames\\cup34.csv"
cup34.to_csv(cup34_df, index=False)

cup1_df = "C:\\Users\\kijah\\Documents\\Data_Bootcamp\\Estimator_Project\\PreprocessedDataFrames\\cup1.csv"
cup1.to_csv(cup1_df, index=False)

con12_df = "C:\\Users\\kijah\\Documents\\Data_Bootcamp\\Estimator_Project\\PreprocessedDataFrames\\con12.csv"
con12.to_csv(con12_df, index=False)

con34_df = "C:\\Users\\kijah\\Documents\\Data_Bootcamp\\Estimator_Project\\PreprocessedDataFrames\\con34.csv"
con34.to_csv(con34_df, index=False)

con1_df = "C:\\Users\\kijah\\Documents\\Data_Bootcamp\\Estimator_Project\\PreprocessedDataFrames\\con1.csv"
con1.to_csv(con1_df, index=False)

rom142_df = "C:\\Users\\kijah\\Documents\\Data_Bootcamp\\Estimator_Project\\PreprocessedDataFrames\\rom142.csv"
rom142.to_csv(rom142_df, index=False)

rom143_df = "C:\\Users\\kijah\\Documents\\Data_Bootcamp\\Estimator_Project\\PreprocessedDataFrames\\rom143.csv"
rom143.to_csv(rom143_df, index=False)

rom122_df = "C:\\Users\\kijah\\Documents\\Data_Bootcamp\\Estimator_Project\\PreprocessedDataFrames\\rom122.csv"
rom122.to_csv(rom122_df, index=False)

rom123_df = "C:\\Users\\kijah\\Documents\\Data_Bootcamp\\Estimator_Project\\PreprocessedDataFrames\\rom123.csv"
rom123.to_csv(rom123_df, index=False)

wir4_df = "C:\\Users\\kijah\\Documents\\Data_Bootcamp\\Estimator_Project\\PreprocessedDataFrames\\wir4.csv"
wir4.to_csv(wir4_df, index=False)

wir6_df = "C:\\Users\\kijah\\Documents\\Data_Bootcamp\\Estimator_Project\\PreprocessedDataFrames\\wir6.csv"
wir6.to_csv(wir6_df, index=False)

wir10_df = "C:\\Users\\kijah\\Documents\\Data_Bootcamp\\Estimator_Project\\PreprocessedDataFrames\\wir10.csv"
wir10.to_csv(wir10_df, index=False)